In [95]:
%pylab inline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

Populating the interactive namespace from numpy and matplotlib


In [96]:
df = pd.read_csv('../data/raw/SUSY.csv',header=None)

In [97]:
df2 = df[0:100]

In [98]:
df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.0,0.972861,0.653855,1.176225,1.157156,-1.739873,-0.874309,0.567765,-0.175000,0.810061,-0.252552,1.921887,0.889637,0.410772,1.145621,1.932632,0.994464,1.367815,0.040714
1,1.0,1.667973,0.064191,-1.225171,0.506102,-0.338939,1.672543,3.475464,-1.219136,0.012955,3.775174,1.045977,0.568051,0.481928,0.000000,0.448410,0.205356,1.321893,0.377584
2,1.0,0.444840,-0.134298,-0.709972,0.451719,-1.613871,-0.768661,1.219918,0.504026,1.831248,-0.431385,0.526283,0.941514,1.587535,2.024308,0.603498,1.562374,1.135454,0.180910
3,1.0,0.381256,-0.976145,0.693152,0.448959,0.891753,-0.677328,2.033060,1.533041,3.046260,-1.005285,0.569386,1.015211,1.582217,1.551914,0.761215,1.715464,1.492257,0.090719
4,1.0,1.309996,-0.690089,-0.676259,1.589283,-0.693326,0.622907,1.087562,-0.381742,0.589204,1.365479,1.179295,0.968218,0.728563,0.000000,1.083158,0.043429,1.154854,0.094859


In [99]:
y = df2[0]
X = df2.drop(0,axis=1)

In [100]:
m = SVC()
str(type(m))

"<class 'sklearn.svm.classes.SVC'>"

In [104]:
class Active_Learner:
    def __init__(self, model, start_size=0.1,end_size=0.25,step_size=0.01):
        if type(model) == type(SVC()):
            self.m = SVC(probability=True)
        else:
            self.m = model
        self.start_size = start_size
        self.end_size = end_size
        self.step_size = step_size
        
    def fit(self, X, y):
        X_train, X_unlabeled, y_train, y_unlabeled = train_test_split(X, y, train_size=0.10, random_state=42)
        
        while(len(y_train) <= len(y)*self.end_size):
            self.m.fit(X_train,y_train)
            y_unlabeled_hat = self.m.predict_proba(X_unlabeled)
            
            #select points with low confidence (class probabilty close to 0.5)
            lowest_conf_idx = np.flip(np.argsort(abs(0.5-y_unlabeled_hat[:,0])),axis=0)
            
            #add points of least confidence to training set
            X_train = np.concatenate((X_train,X_unlabeled[:int(len(y)*self.step_size)]),axis=0)
            y_train = np.concatenate((y_train,y_unlabeled[:int(len(y)*self.step_size)]),axis=0)
            
            #remove these points from "unlabeled" set
            mask = np.ones(len(y_unlabeled), dtype=bool)
            mask[lowest_conf_idx[0:int(len(y)*self.step_size)]] = False
            y_unlabeled = y_unlabeled[mask]
            X_unlabeled = X_unlabeled[mask]
            
    def predict(self,X):
        return self.m.predict(X)

In [105]:
sv = SVC()
al = Active_Learner(sv)
al.fit(X,y)

/Users/robbieculkin/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [107]:
from sklearn.metrics import classification_report

print (classification_report(y,al.predict(X)))

             precision    recall  f1-score   support

        0.0       0.76      0.73      0.75        60
        1.0       0.62      0.65      0.63        40

avg / total       0.70      0.70      0.70       100

